In [1]:
from konlpy.tag import Okt
okt = Okt()

In [2]:
# 모델 불러오기(학습데이터: 네이버 영화 평가)
from keras.models import load_model
model = load_model('movie_review_model.h5')

In [3]:
def tokenize(doc):
    #형태소와 품사를 join
    return ['/'.join(t) for t in okt.pos(doc, norm=True, stem=True)]

In [4]:
import pandas as pd
train_df = pd.read_csv("ratings_train.txt", "\t")

In [5]:
train_df.isnull().any() #document에 null값이 있다.
train_df['document'] = train_df['document'].fillna(''); #null값을 ''값으로 대체

In [6]:
#tokenize 과정은 시간이 오래 걸릴수 있음...
train_docs = [(tokenize(row[1]), row[2]) for row in train_df.values]

In [7]:
tokens = [t for d in train_docs for t in d[0]]
print("토큰개수:", len(tokens))

토큰개수: 2159921


In [14]:
import nltk
import numpy as np
text = nltk.Text(tokens, name='NMSC')

In [29]:
FREQUENCY_COUNT = 100; #시간적 여유가 있다면 100개를 해보도록~
selected_words = [f[0] for f in text.vocab().most_common(FREQUENCY_COUNT)]

In [30]:
#영화평가에서 사용된 단어리스트 문서에서 상위 100개들중 포함되는 단어들이 개수
def term_frequency(doc):
    return [doc.count(word) for word in selected_words]

In [31]:
def predict_sentiment(review):
    token = tokenize(review)
    tfq = term_frequency(token)
    data = np.expand_dims(np.asarray(tfq).astype('float32'), axis=0)
    score = float(model.predict(data))
    if(score > 0.5):
        print(f"{review} ==> 긍정 ({round(score*100)}%)")
    else:
        print(f"{review} ==> 부정 ({round((1-score)*100)}%)")

In [32]:
predict_sentiment("재미 정말 없어요")

재미 정말 없어요 ==> 부정 (92%)


In [33]:
predict_sentiment("괜찮")

괜찮 ==> 긍정 (56%)


In [34]:
predict_sentiment("카카오 주가 전망 오히려 지금이 매력적인 이유")

카카오 주가 전망 오히려 지금이 매력적인 이유 ==> 긍정 (75%)


In [35]:
predict_sentiment("슈퍼개미가 카카오를 아직 팔지 않은 이유 (전업투자자 남석관)")

슈퍼개미가 카카오를 아직 팔지 않은 이유 (전업투자자 남석관) ==> 부정 (61%)


In [36]:
predict_sentiment("청약 첫날 12조 몰린 카뱅…'따상' 커녕 반토막 우려, 왜?/ 박해린의 뉴스&마켓 /한국경제TV뉴스")

청약 첫날 12조 몰린 카뱅…'따상' 커녕 반토막 우려, 왜?/ 박해린의 뉴스&마켓 /한국경제TV뉴스 ==> 부정 (69%)


In [37]:
predict_sentiment("[카카오 주가] 카카오 카카오뱅크 2,600조 청약! 외국인, 기관 싹슬이 20만원 돌파 충분하다!")

[카카오 주가] 카카오 카카오뱅크 2,600조 청약! 외국인, 기관 싹슬이 20만원 돌파 충분하다! ==> 긍정 (74%)
